In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


dataset = pd.read_csv('train.csv')

In [5]:
test_dataset = pd.read_csv('test.csv')

In [5]:
#dataset.head()


In [6]:
#print(dataset.shape, test_dataset.shape)

In [6]:
dataset_ = dataset.iloc[:, :].values

In [7]:
test_dataset_ = test_dataset.iloc[:,:].values

In [8]:
target = dataset_[:,4]
target=target.astype('int')
data = dataset_[:,:-1]
data[:,0] = data[:,0].astype('int')
data[:,3] = data[:,3].astype('int')

print(data.shape, target.shape)

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.35, random_state = 42)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(489517, 4) (489517,)
(318186, 4) (318186,) (171331, 4) (171331,)


In [ ]:
#X_train_pd = pd.DataFrame(X_train)

In [ ]:
#X_train_pd.describe()

In [ ]:
#X_train_pd.head()

Токенизируем текст и переведем все символы в нижний регистр.

In [9]:
from nltk.tokenize import TweetTokenizer
tokenizer = TweetTokenizer()
preprocess = lambda text: ' '.join(tokenizer.tokenize(text.lower()))

In [10]:
for i in range(len(X_train)):
    X_train[i,1] = preprocess(X_train[i,1])
    X_train[i,2] = preprocess(X_train[i,2])

In [11]:
for i in range(len(X_test)):
    X_test[i,1] = preprocess(X_test[i,1])
    X_test[i,2] = preprocess(X_test[i,2])

In [12]:

for i in range(len(X_test)):
    test_dataset_[i,1] = preprocess(test_dataset_[i,1])
    test_dataset_[i,2] = preprocess(test_dataset_[i,2])

Создадим матрицы TF-IDF для двух текстовых столбцов поотдельности и потом соединим их в одну матрицу. К ней добавим нормализованный столбец цен.

In [13]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [14]:
tfidf_vectorizer = TfidfVectorizer()

title_train_tfidf = tfidf_vectorizer.fit_transform(X_train[:,1])
title_test_tfidf = tfidf_vectorizer.transform(X_test[:,1])
title_dataset = tfidf_vectorizer.transform(test_dataset_[:,1])

description_train_tfidf = tfidf_vectorizer.fit_transform(X_train[:,2])
description_test_tfidf = tfidf_vectorizer.transform(X_test[:,2])
description_dataset = tfidf_vectorizer.transform(test_dataset_[:,2])

In [15]:
from scipy.sparse import csr_matrix
import scipy as sp

from sklearn import preprocessing

train_1 = np.array(X_train[:,3].astype(float)).reshape(-1,1)
test_1 = np.array(X_test[:,3].astype(float)).reshape(-1,1)
dataset_1 = np.array(test_dataset_[:,3].astype(float)).reshape(-1,1)

train_ = preprocessing.normalize(train_1, axis = 0)
test_ = preprocessing.normalize(test_1, axis = 0)
dataset_ = preprocessing.normalize(dataset_1, axis = 0)

#X_train_counts = sp.sparse.hstack(( title_train_counts, description_train_counts,csr_matrix(train_) ))
#X_test_counts = sp.sparse.hstack(( title_test_counts, description_test_counts,csr_matrix(test_) ))

X_train_tfidf = sp.sparse.hstack(( title_train_tfidf, description_train_tfidf,csr_matrix(train_) ))
X_test_tfidf = sp.sparse.hstack(( title_test_tfidf, description_test_tfidf,csr_matrix(test_) ))
test_dataset_tdidf = sp.sparse.hstack(( title_dataset, description_dataset,csr_matrix(dataset_) ))

Обучим на получившейся матрице обычную логрегрессию и посчитаем accuracy.

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(C=7.0, solver = 'saga', 
                         multi_class='multinomial', n_jobs=-1, random_state=40)
lr.fit(X_train_tfidf, y_train)

y_predicted_counts = lr.predict(X_test_tfidf)

In [40]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_predicted_counts)
print(accuracy)

0.8805995412388885


На всякий случай посчитаем точность разными способами

In [41]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report


precision = precision_score(y_test, y_predicted_counts, pos_label=None,average='weighted')             
recall = recall_score(y_test, y_predicted_counts, pos_label=None, average='weighted')
f1 = f1_score(y_test, y_predicted_counts, pos_label=None, average='weighted')
accuracy = accuracy_score(y_test, y_predicted_counts)

print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))

accuracy = 0.881, precision = 0.880, recall = 0.881, f1 = 0.880


Предскажем категории на тестовых данных и запишем их в файл result1.csv

In [42]:
y = lr.predict(test_dataset_tdidf)
y = np.array(y).reshape(-1,1)
id_ = np.array(test_dataset_[:,0]).reshape(-1,1)

result1 = np.hstack((id_,y))

In [43]:
#df.to_csv('result1.csv', sep='\t')
import csv
 
FILENAME = "result1.csv"

with open(FILENAME, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerows(result1)
     

Разберемся с иерархией категорий, для этого создадим матрицу, в которой каждый столбец отвечает одному уровню иерархии.

In [44]:
category = pd.read_csv('category.csv')

In [45]:
category = category.iloc[:, :].values

In [46]:
y1 = []

for s in category[:,1]:
    if s.find('Бытовая электроника') == 0:
        y1.append(0)
    elif  s.find('Для дома и дачи') == 0:
        y1.append(1)
    elif  s.find('Личные вещи') == 0:
        y1.append(2)
    elif  s.find('Хобби и отдых') == 0:
        y1.append(3)
    else :
        y1.append(-1) 

In [47]:
y2 = []

for s in category[:,1]:
    if s.find('Бытовая электроника|Телефоны') == 0:
        y2.append(0)
    elif  s.find('Бытовая электроника|Игры, приставки и программы') == 0:
        y2.append(1)
    elif  s.find('Бытовая электроника|Аудио и видео') == 0:
        y2.append(2)
    elif  s.find('Бытовая электроника|Товары для компьютера') == 0:
        y2.append(3)
    elif  s.find('Для дома и дачи|Ремонт и строительство') == 0:
        y2.append(4)
    elif  s.find('Для дома и дачи|Мебель и интерьер') == 0:
        y2.append(5)
    elif  s.find('Для дома и дачи|Бытовая техника') == 0:
        y2.append(6)
    elif  s.find('Личные вещи|Товары для детей и игрушки') == 0:
        y2.append(7)
    elif  s.find('Личные вещи|Одежда, обувь, аксессуары') == 0:
        y2.append(8)
    elif  s.find('Личные вещи|Часы и украшения') == 0:
        y2.append(9)
    elif  s.find('Хобби и отдых|Спорт и отдых') == 0:
        y2.append(10)
    elif  s.find('Хобби и отдых|Книги и журналы') == 0:
        y2.append(11)
    elif  s.find('Хобби и отдых|Коллекционирование') == 0:
        y2.append(12)
    elif  s.find('Хобби и отдых|Музыкальные инструменты') == 0:
        y2.append(13)
    else :
        y2.append(-1) 


In [48]:
y3 = []

for s in category[:,1]:
    if s.find('Личные вещи|Одежда, обувь, аксессуары|Женская одежда') == 0:
        y3.append(0)
    else :
        y3.append(-1) 

In [49]:
y1 = np.array(y1).reshape(-1,1)
y2 = np.array(y2).reshape(-1,1)
y3 = np.array(y3).reshape(-1,1)

transl = np.hstack((y1,y2,y3))

Посчитаем точность предсказания на 1 (самом низком) уровне иерархии категорий.

In [50]:
n = len(y_test)
y_1  = np.zeros(n)
y_pred_1 = np.zeros(n)

for i in range(n):
    if transl[y_test[i],0] != -1:
        y_1[i] = transl[y_test[i],0]
        y_pred_1[i] = transl[y_predicted_counts[i],0]
    else:
        y_1[i] = y_test[i]
        y_pred_1[i] = y_predicted_counts[i]
print(accuracy_score(y_1, y_pred_1))

0.9622952063549504


На втором уровне:

In [51]:
n = len(y_test)
y_2  = np.zeros(n)
y_pred_2 = np.zeros(n)

for i in range(n):
    if transl[y_test[i],1] != -1:
        y_2[i] = transl[y_test[i],1]
        y_pred_2[i] = transl[y_predicted_counts[i],1]
    else:
        y_2[i] = y_test[i]
        y_pred_2[i] = y_predicted_counts[i]
    
print(accuracy_score(y_2, y_pred_2))

0.9427307375781382


На третьем:

In [52]:
n = len(y_test)
y_3  = np.zeros(n)
y_pred_3 = np.zeros(n)

for i in range(n):
    if transl[y_test[i],2] != -1:
        y_3[i] = transl[y_test[i],2]
        y_pred_3[i] = transl[y_predicted_counts[i],2]
    else:
        y_3[i] = y_test[i]
        y_pred_3[i] = y_predicted_counts[i]
    
print(accuracy_score(y_3, y_pred_3))

0.8848719729646124


А теперь попробуем Word2Vec. Для этого сначала подготовим данные.

In [32]:
all_data_train = dataset.apply(lambda x: x['title'] + ' . ' + x['description'], 1)
all_data_test = test_dataset.apply(lambda x: x['title'] + ' . ' + x['description'], 1)


In [33]:
train_proc = all_data_train.apply(lambda x: preprocess(x), 1)
test_proc = all_data_train.apply(lambda x: preprocess(x), 1)

In [34]:
dataset['tokens'] = train_proc.apply(lambda x: x.split(), 1)
test_dataset['tokens'] = test_proc.apply(lambda x: x.split(), 1)

In [35]:
import nltk

all_words = []

for s in train_proc:
    sentences = nltk.sent_tokenize(s)
    all_words += [sent.split() for sent in sentences]

In [39]:
for s in test_proc:
    sentences = nltk.sent_tokenize(s)
    all_words += [sent.split() for sent in sentences]


In [20]:
from gensim.models import Word2Vec

word2vec = Word2Vec(all_words, size = 200, min_count=1)

In [21]:
def get_average_word2vec(tokens_list, vector, generate_missing=False, k=300):
    if len(tokens_list)<1:
        return np.zeros(k)
    if generate_missing:
        vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
    else:
        vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged

def get_word2vec_embeddings(vectors, data, generate_missing=False):
    embeddings = data['tokens'].apply(lambda x: get_average_word2vec(x, vectors, 
                                                                                generate_missing=generate_missing))
    return list(embeddings)

In [49]:
from sklearn import preprocessing

embeddings = get_word2vec_embeddings(word2vec, dataset)

price = np.array(dataset['price'].astype(float)).reshape(-1,1)
price_ = preprocessing.normalize(price, axis = 0)
embeddings_ = np.hstack((embeddings, price_))
embeddings_ = np.array(embeddings_)

In [60]:

X_train_word2vec, X_test_word2vec, y_train_word2vec, y_test_word2vec = train_test_split(embeddings_, target, 
                                                                                        test_size=0.2, random_state=40)
print(X_train_word2vec.shape, y_train_word2vec.shape, X_test_word2vec.shape, y_test_word2vec.shape)

(391613, 201) (391613,) (97904, 201) (97904,)


In [61]:
embeddings_test = get_word2vec_embeddings(word2vec, test_dataset)

price_test = np.array(test_dataset['price'].astype(float)).reshape(-1,1)
price_test_ = preprocessing.normalize(price_test, axis = 0)

embeddings_test_ = np.hstack((embeddings_test, price_test_))
embeddings_test_ = np.array(embeddings_test_)

In [62]:
from sklearn.linear_model import LogisticRegression

lr_w2v = LogisticRegression(C=30.0, solver = 'saga', 
                         multi_class='multinomial', n_jobs=-1, random_state=40)
lr_w2v.fit(X_train_word2vec, y_train_word2vec)

y_pred_w2v = lr_w2v.predict(X_test_word2vec)

In [68]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
precision = precision_score(y_test_word2vec, y_pred_w2v, pos_label=None,average='weighted')             
recall = recall_score(y_test_word2vec, y_pred_w2v, pos_label=None, average='weighted')
f1 = f1_score(y_test_word2vec, y_pred_w2v, pos_label=None, average='weighted')
accuracy = accuracy_score(y_test_word2vec, y_pred_w2v)
#print(classification_report(y_test_word2vec, y_pred_w2v))

print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))

accuracy = 0.824, precision = 0.819, recall = 0.824, f1 = 0.821


In [69]:
y = lr_w2v.predict(embeddings_test_)
y = np.array(y).reshape(-1,1)
id_ = np.array(test_dataset_[:,0]).reshape(-1,1)

result2 = np.hstack((id_,y))

In [71]:
import csv
 
FILENAME = "result2.csv"

with open(FILENAME, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerows(result2)

Ну так как тут точность меньше 88, то по уровням я решила не делать . 